<img src="images/logos/The-Construct-logo-new.png" width="700">

# PRESENTS...

# ROS Developers Open Class n.163

<img src="images/OpenClass163.jpeg" width="650" />

## How to Launch the Simulation

To start the simulation, first we'll need to source our workspace:

- Open a terminal window by clicking on the shell icon on the bottom left side of your screen:

<img src="images/shell-superapp.png" width="450" />

- Copy and paste the following terminal commands in your shell:


<span class="badge badge-pill badge-primary">
    <i class="fa fa-play"></i>
    &nbsp;
    Execute in Shell
</span>

In [ ]:
cd ~/ros2_ws/src
./start_turtlebot3_multi.sh

And that's it! You should be able to see the simulation and control everything as if it was the real robot if you go to the Gazebo button in the bottom left side of your screen:

<img src="images/gazebo-icon.png" width="100" />


**Wait around 30 seconds maximum** for the simulaion to start and you should see this simulation now:


<img src="images/multi-nav.png" width="800" />

<div class="jumbotron m-0">
    <hr />
    <h1 class="text-center">
        <span class="text-primary">
            Multi-robot Navigation
        </span>
    </h1>
    <hr />
</div>

## Configuring the robots for multi-robot environments

If you have a multi-robot environment, each robot will likely have the same `/cmd_vel` topic name and the same `base_footprint` frame name, and even the same control node name. 

This is not correct for multi-robot environments because the navigation algorithm cannot identify which robot it is talking to.

So first, correctly configure your robots, and bringup separating *topics*, *frames*, and *node names* into `namespaces`. **THIS IS NOT THE SUBJECT FOR THIS COURSE**. So, for the rest of this unit, you will assume that `namespaces` properly separate your robots. 

The `namespace` is the label that you will assign to each robot (for example, tb3_0, tb3_1, etc.). So, each robot ›may have the same topics, frames, and node names, but all of them will start with a unique `namespace` that identifies that robot.

The configuration of robots for proper usage of the `namespaces` is done at:
1. **In simulated robots**: at the URDF description file and the Gazebo plugins
2. **In real robots**: at the bringup routine, provided by the vendor

If the URDF or plugins are not correctly created to allow `namespaces`, or if the bringup is not properly created, you can have difficulty making your robots boot in `namespaces` mode.

**IMPORTANT**: It is mandatory that your robots already work separated by `namespaces`. In this course, we did it for you. However, if you want to apply this unit to your own robot, you can do that job yourself. 

Ensure that each of your robots has a specific name for:
* Topics
* Frames 
* Nodes

You may need to touch the URDF model of your robot and the configuration of your drivers (for sensors and actuators), to include the namespaces in them.

In this unit, **the simulation is properly set up with two TurtleBot3 robots, one named tb3_0 and the other tb3_1**. Check it.

#### Check the node names

Make a list of the nodes running:

In [ ]:
ros2 node list

You should get something like this:

In [ ]:
/gazebo
/tb3_0/robot_state_publisher
/tb3_0/turtlebot3_diff_drive
/tb3_0/turtlebot3_imu
/tb3_0/turtlebot3_joint_state
/tb3_0/turtlebot3_laserscan
/tb3_1/robot_state_publisher
/tb3_1/turtlebot3_diff_drive
/tb3_1/turtlebot3_imu
/tb3_1/turtlebot3_joint_state
/tb3_1/turtlebot3_laserscan

As you can see, each node related to a robot starts with the `namespace` of that robot.

#### Check the topic names

Make a listing of the topics:

In [ ]:
ros2 topic list

You should get the following:

In [ ]:
/clock
/parameter_events
/performance_metrics
/rosout
/tb3_0/cmd_vel
/tb3_0/imu
/tb3_0/joint_states
/tb3_0/odom
/tb3_0/robot_description
/tb3_0/scan
/tb3_1/cmd_vel
/tb3_1/imu
/tb3_1/joint_states
/tb3_1/odom
/tb3_1/robot_description
/tb3_1/scan
/tf
/tf_static

As you can see, each robot has a different IMU, odom, scan, camera, and joint states topic, separated by the robot `namespace`.

<div class="bg-info text-center">
    - Notes -
</div>

To move the robots around, you can use the following commands:

In [ ]:
ros2 run teleop_twist_keyboard teleop_twist_keyboard --ros-args --remap cmd_vel:=/tb3_0/cmd_vel

In [ ]:
ros2 run teleop_twist_keyboard teleop_twist_keyboard --ros-args --remap cmd_vel:=/tb3_1/cmd_vel

<div class="bg-info text-center">
    - End of Notes -
</div>

#### Check the frame names

Print the full TF for this ROS2 system with two robots.

For that, use the following command:

In [ ]:
cd ~/ros2_ws/src; ros2 run tf2_tools view_frames

That will generate a `frames.pdf` file inside the directory `/home/user/ros2_ws/src`. Download that pdf using the IDE *Download* option, and open it. It should look like the following:

![](images/two_robots_tf_v2.png)

## Configuring a navigation system for each robot

In order to have all the robots autonomously moving, **you need to configure and launch a complete navigation system FOR EACH ROBOT**. This means, one `localization` system, one `controller_server`, one `planner_server`, one `bt-navigator`, etc., for each robot. 

The only one that will be common to all the robots is the `map_server`. **There will be a single `map_server` for all the robots** because all the robots will use the same map. So start by launching it.

### Multi-robot localization

Now, launch a `localization` system for each robot. This means launching two `amcl` nodes, each one configured appropriately for the robot it must operate.

<div class="bg-danger text-center">
    - Exercise 6.2 -
</div>

#### Step 1. Add a `namespace` to the launches of `amcl` nodes

* You need to launch two `amcl` nodes.
* To each node launch, add a `namespace` argument with the `namespace` of the robot to which that `amcl` node corresponds.

**IMPORTANT: By adding a namespace, the node launched will modify all its topics, node name, and services, appending the `namespace` to the beginning of the names.**

**IMPORTANT 2: the frames indicated in the config file will not be automatically modified by adding a `namespace` in the `launch` file. Those need to be manually modified on the config file itself (see below)**

Example (for the `tb3_0` robot):

In [ ]:
Node(
            namespace='tb3_0',
            package='nav2_amcl',
            executable='amcl',
            name='amcl',
            output='screen',
            parameters=[tb3_0_config]
        ),

The node launched with that code will not be named `amcl`. Instead, it will be named `tb3_0/amcl`. The same will happen with its topics and services.

* Inside the `launch` directory of the `localization_server` package, create another launch file named `multi_localization.launch.py` with the same content as the previous launch file.
* Modify the new launch to now launch **TWO** `amcl` nodes, each with a different `namespace` according to the robot it will localize.
* Remember to modify each node's `parameters` argument to point to **TWO** different config files.

#### Step 2. Create a specific config file for each robot 

* Create two copies of the `amcl_config.yaml` file, each one with the name of the robot that belongs to, at the beginning of the name. Ex. `tb3_0_amcl_config.yaml`.
* Modify the content of each config file to accommodate its robot. You have to modify the following:
    1. The first line must contain the **full new name** of the node.
    2. All frames must be modified to include the `namespace`. The only frame that does not have to be changed is the `global_frame_id` because there is a single global frame for all the robots (the one on the map).
    3. Topics do not need to be modified because they are automatically modified by the `namespace` argument of the launch file. This does not apply to frames.
    4. The `map_topic` parameter must be forced to `/map`. If you do not force it with the `/`, it automatically connects to `tb3_0/map`, which is not the topic that the map server is publishing.
* Add two config file lines in the `multi_localization.launch.py`, one for each `amcl` config.
* Assign the proper config file to the appropriate `amcl` `parameters` field.

Example for `tb3_0`:

<p style="background:green;color:white;">tb3_0_amcl_config.yaml</p>

In [ ]:
tb3_0/amcl:
  ros__parameters:
    use_sim_time: True
    alpha1: 0.2
    alpha2: 0.2
    alpha3: 0.2
    alpha4: 0.2
    alpha5: 0.2
    base_frame_id: "tb3_0/base_footprint"
    beam_skip_distance: 0.5
    beam_skip_error_threshold: 0.9
    beam_skip_threshold: 0.3
    do_beamskip: false
    global_frame_id: "map"
    lambda_short: 0.1
    laser_likelihood_max_dist: 2.0
    laser_max_range: 100.0
    laser_min_range: -1.0
    laser_model_type: "likelihood_field"
    max_beams: 60
    max_particles: 8000
    min_particles: 200
    odom_frame_id: "tb3_0/odom"
    pf_err: 0.05
    pf_z: 0.99
    recovery_alpha_fast: 0.0
    recovery_alpha_slow: 0.0
    resample_interval: 1
    robot_model_type: "differential"
    save_pose_rate: 0.5
    sigma_hit: 0.2
    tf_broadcast: true
    transform_tolerance: 1.0
    update_min_a: 0.2
    update_min_d: 0.25
    z_hit: 0.5
    z_max: 0.05
    z_rand: 0.5
    z_short: 0.05
    scan_topic: "scan"
    map_topic: "/map"
    # ACTIVATE THE set_initial_pose WHEN YOU HAVE A PROPER initial_pose, by uncommenting the code below
    #set_initial_pose: true
    #initial_pose:
    # x: 7.778
    # y: -9.589
    # z: 0.0
    # a: -0.211

tb3_0/amcl_map_client:
  ros__parameters:
    use_sim_time: True

tb3_0/amcl_rclcpp_node:
  ros__parameters:
    use_sim_time: True

#### Step 3. Add the nodes to the lifecycle manager

In the launch file, remember to tell the `lifecycle_manager` to launch the **TWO** `amcl` nodes. You need to provide **the full node names** with their `namespaces`.

In [ ]:
Node(
    package='nav2_lifecycle_manager',
    executable='lifecycle_manager',
    name='lifecycle_manager_localization',
    output='screen',
    parameters=[{'use_sim_time': True},
                {'autostart': True},
                {'bond_timeout':0.0},
                {'node_names': ['map_server', 'tb3_0/amcl', 'tb3_1/amcl']}]
)

**IMPORTANT:** Please notice that you are adding a new parameter `{'bond_timeout':0.0},` required to prevent launching errors.

#### Step 4. Modify the `map_server` launch

To prevent errors, specify the `topic_name` and `frame_id` of the `map_server`. Unfortunately, those two parameters were not configured when you did the single robot `map_server`.

In [ ]:
Node(
    package='nav2_map_server',
    executable='map_server',
    name='map_server',
    output='screen',
    parameters=[{'use_sim_time': True}, 
                {'topic_name':"map"},
                {'frame_id':"map"},
                {'yaml_filename':map_file}]
),

Also, remember to change the `map_file` to which the robot is pointing.

<p style="background:green;color:white;">multi_localization.launch.py</p>

In [ ]:
import os

from ament_index_python.packages import get_package_share_directory
from launch import LaunchDescription
from launch_ros.actions import Node

def generate_launch_description():
    
    tb3_0_config = os.path.join(get_package_share_directory('localization_server'), 'config', 'tb3_0_amcl_config.yaml')
    tb3_1_config = os.path.join(get_package_share_directory('localization_server'), 'config', 'tb3_1_amcl_config.yaml')

    map_file = os.path.join(get_package_share_directory('map_server'), 'config', 'turtlebot_area.yaml')

    return LaunchDescription([
        Node(
            package='nav2_map_server',
            executable='map_server',
            name='map_server',
            output='screen',
            parameters=[{'use_sim_time': True}, 
                        {'topic_name':"map"},
                        {'frame_id':"map"},
                        {'yaml_filename':map_file}]
        ),
            
        Node(
            namespace='tb3_0',
            package='nav2_amcl',
            executable='amcl',
            name='amcl',
            output='screen',
            parameters=[tb3_0_config]
        ),

        Node(
             namespace='tb3_1',
             package='nav2_amcl',
             executable='amcl',
             name='amcl',
             output='screen',
             parameters=[tb3_1_config]
        ),

        Node(
            package='nav2_lifecycle_manager',
            executable='lifecycle_manager',
            name='lifecycle_manager_localization',
            output='screen',
            parameters=[{'use_sim_time': True},
                        {'autostart': True},
                        {'bond_timeout':0.0},
                        {'node_names': ['map_server', 'tb3_0/amcl', 'tb3_1/amcl']}]
        )
    ])

#### Step 5. Launch the new multi-robot localization

* Compile, source and launch
* You should get a message indicating that both `amcl` nodes are waiting for the initial position.
* You will use RVIZ to initialize the robots.

#### Step 6. Configure RVIZ for multi-robot

Launch RVIZ

In [ ]:
rviz2

* Load the RVIZ configuration of the localization unit.
* Modify the topic of the `LaserScan` to point to the `tb3_0/scan` topic. Indicate `Color Transformer` as `FlatColor` and then assign to it a red color.
* Add another `LaserScan` display and make it point to the `tb3_1/scan` topic. Indicate `Color Transformer` as `FlatColor` and then assign to it a blue color.
* Modify the `RobotModel` display as follows:
    * In the `Description Source`, select `Topic`.
    * In the `Description Topic`, indicate the topic `/tb3_0/robot_description`. That is the topic that is publishing the robot model.
    * In the `TF prefix`, indicate the namespace of the robot. For tb3_0, indicate `tb3_0`.
* Add another `RobotModel` and configure it the same way for the `tb3_1` robot.
* Add the `Panel` of `Tool Properties`. You will modify the topics that send `2D Pose Estimate` and `2D Goal Pose` to the different robots.
* In the `Tool Properties`, select the `2D Pose Estimate` topic and write the one for the tb3_1 robot `/tb3_1/initialpose`.

![](images/init_pose_topic.png)

* Then click on the button of the `2D Pose Estimate` and provide an initial position for the first robot (the `tb3_1`)
* Repeat the `2D Pose Estimate` process for `tb3_0`.
* Now both robots must be localized.

![](images/two_robots_localized_v2.png)

**IMPORTANT**: Remember to save this RVIZ configuration.

Check how the localization of both robots has modified the TF. Run the following command:

In [ ]:
ros2 run tf2_tools view_frames

The TF must now show the connection between the two robot branches. Each `amcl` node has published a transform between the `map` and the `odom`frame (`tb3_0/odom` and `tb3_1/odom`).

![](images/global_tf_localized.png)

## Remember you can learn more about ROS2 Navigation in our course in the Academy:

<a href="https://app.theconstructsim.com/Course/109" target="_blank">ROS2 Navigation</a>

<a href="https://app.theconstructsim.com/Course/109" target="_blank"><img src="images/ros2-nav-gala.png" width="" /></a>

# Build the future, Become a ROS DEVELOPER!